In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib.patches import Polygon
from matplotlib.patches import Circle
from mpl_toolkits.axes_grid1 import make_axes_locatable
import imutils
import glob
import cv2  # importing cv
import find_peak
import scipy.special as special
from matplotlib import ticker
from scipy.fft import fft, fftfreq
from scipy.optimize import curve_fit
import nfft
import scipy
import scipy.fft
import functions_analysis as fa
from skimage import img_as_float
from skimage import filters
%matplotlib inline 

path = r'C:\Path\To\Data'
path_main=path+r'\PROCESSED'
scan_number = [ '800nm',  '400nm' ]
scan_name = [ '1.55 eV',  '3.1 eV' ]


path=[]
df_list = []
imgON=[]
imgOFF=[]
delay=[]


for scan in scan_number:
    path.append(path_main + r'\PROC_'+scan)
    df = pd.read_pickle(path_main + r'\PROC_'+scan, compression={'method': 'gzip', 'compresslevel': 1, 'mtime': 1})
    imgOFF.append(np.stack(df['imgOFF']))
    imgON.append(np.stack(df['imgON']))
    delay.append(np.stack(df['delay']))
    df_list.append(df)

print(delay)


In [2]:
cc = []
BP_pos = []
BP_name = []
BP_name = ['{100}', '{200}', '{110}']
DS_pos = []
DS_name = [r'$\Gamma \to K$ 1o', r'$\Gamma \to K$ 2o', r'$\Gamma \to M$ 1o', r'$\Gamma \to M$ 2o']
DS_name = [r'$K$', r'$\Gamma \to M$']

Zorder=[]
cal = []
for df in df_list:
    L = df.attrs['peak_dist']
    Zo = df.attrs['Zorder_pos']
    Zorder.append(Zo)
    cal.append(df.attrs['calibration'])
    t0 = df.attrs['t0_index']
    cc.append( L * np.sqrt(3)/3 *np.exp(1j * np.pi  * ( 1./3 * np.arange(7)))) ## for hexagonal BZ
    #### construction position of the Bragg peak, ordered by family

    BP_pos.append(fa.bragg_peak_position(Zo, L))

    #### construction position of the K-pont, ordered by family
    DS_pos.append(fa.diffuse_scattering_position2(Zo, L))


In [ ]:
### 2d voigt

o=1
# select 1st order Bragg peak
print(BP_pos[o][0][0])
ROI = []
for o in [0,1]:
    ROI.append(find_peak.ROI_around(BP_pos[o][0][0], 15))
rot=False
result_off_all = []
result_on_all = []
for o in [0,1]:
    result_on = []
    for imON in imgON[o]:
        result_on.append(find_peak.fit_2dvoigt(imON, ROI[o], show_plot=False, rotation=rot))
    result_on_all.append(result_on)
    result_off = [] 
    for imOFF in imgOFF[o]:
        result_off.append(find_peak.fit_2dvoigt(imOFF, ROI[o], show_plot=False, rotation=rot))
    result_off_all.append(result_off)

print(result_on[0])

In [ ]:
def extract_parameter(param_list, param_name):
    return np.array([param[param_name].value for param in param_list])


def calculate_fwhm_voigt2(sigma_x, sigma_y, gamma_x, gamma_y):
    # Gaussian FWHM
    fwhm_gaussian_x = 2 * np.sqrt(2 * np.log(2)) * sigma_x
    fwhm_gaussian_y = 2 * np.sqrt(2 * np.log(2)) * sigma_y
    
    # Lorentzian FWHM
    fwhm_lorentzian_x = 2 * gamma_x
    fwhm_lorentzian_y = 2 * gamma_y
    
    # Voigt FWHM
    fwhm_voigt_x = (0.5346 * fwhm_lorentzian_x +
                    np.sqrt(0.2166 * fwhm_lorentzian_x**2 + fwhm_gaussian_x**2))
    fwhm_voigt_y = (0.5346 * fwhm_lorentzian_y +
                    np.sqrt(0.2166 * fwhm_lorentzian_y**2 + fwhm_gaussian_y**2))
    
    return fwhm_voigt_x, fwhm_voigt_y
FWHM = {}
FWHM[scan_name[0]] = {}
FWHM[scan_name[1]] = {}


for o, result_on in enumerate(result_on_all):
    f_x_on, f_y_on = (calculate_fwhm_voigt2(extract_parameter(result_on, 'sigmax'), extract_parameter(result_on, 'sigmay'), extract_parameter(result_on, 'gammax'), extract_parameter(result_on, 'gammay')))
    FWHM[scan_name[o]]['f_x_on'] = f_x_on
    FWHM[scan_name[o]]['f_y_on'] = f_y_on

for o, result_off in enumerate(result_off_all):
    f_x_off, f_y_off = (calculate_fwhm_voigt2(extract_parameter(result_off, 'sigmax'), extract_parameter(result_off, 'sigmay'), extract_parameter(result_off, 'gammax'), extract_parameter(result_off, 'gammay')))
    FWHM[scan_name[o]]['f_x_off'] = f_x_off
    FWHM[scan_name[o]]['f_y_off'] = f_y_off

print(f_y_off)

In [7]:
plt.rcParams.update({'font.size': 20})
plt.rc('axes', linewidth=2)
plt.rcParams['xtick.major.size'] = 6
plt.rcParams['xtick.major.width'] = 2
plt.rcParams['xtick.minor.size'] = 3
plt.rcParams['xtick.minor.width'] = 1
plt.rcParams['ytick.major.size'] = 6
plt.rcParams['ytick.major.width'] = 2
plt.rcParams['ytick.minor.size'] = 3
plt.rcParams['ytick.minor.width'] = 1
plt.rc('axes', linewidth=2)
plt.rc('font', family='arial',size=26)
plt.rc('text', usetex=False)

In [ ]:
## along qx and qy 


fig, axis = plt.subplots(1,2, figsize=(12, 6),layout='tight')

end = len(delay)
c = ['r', 'b']

leg_bp = []
leg_scan = []


for o in [0,1]:
    y = FWHM[scan_name[o]]['f_x_on']-FWHM[scan_name[o]]['f_x_off']
    axis[0].plot(delay[o], 1e4*cal[o]*(y-y[0]), 'o-', c=c[o], label=scan_name[o])

axis[0].set_title(r'along $q_x$')

for o in [0,1]:
    y=FWHM[scan_name[o]]['f_y_on']-FWHM[scan_name[o]]['f_y_off']
    axis[1].plot(delay[o], 1e4*cal[o]*(y-y[0]), 'o-', c=c[o] ,label=scan_name[o])
axis[1].set_title(r'along $q_y$')


for a in axis:
    a.set_ylabel('$\Delta$FHWM ($10^{-4}/\AA$)')
    a.set_xlabel('delay (ps)')
    a.legend()
    a.tick_params(axis='both', direction='in', pad = 15)
    a.set_xlim(-3, 10)
    a.set_ylim(-4,1)


In [ ]:
## Figure S5

fig, axis = plt.subplots(1,1, figsize=(7, 7),layout='tight')

end = len(delay)
c_bp = ['k', 'orange', 'g']
c_ds = ['b', 'm', 'y', 'sienna']
s = ['*-', 'o-', '+-']
leg_bp = []
leg_scan = []


for o in [0,1]:
    y = FWHM[scan_name[o]]['f_x_on']-FWHM[scan_name[o]]['f_x_off']
    axis.plot(delay[o], 1e4*cal[o]*(y-y[0]), 'o-', c=c[o], label=scan_name[o])

axis.set_ylabel('$\Delta$FHWM ($10^{-4}/\AA$)')
axis.set_xlabel('delay (ps)')
axis.legend()
axis.tick_params(axis='both', direction='in', pad = 15)
axis.set_xlim(-3, 10)
axis.set_ylim(-4,1)

plt.savefig('./fit_fhwm_x.pdf', format='pdf')